In [1]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio
from electoralyze import region
from electoralyze.common import constants
from electoralyze.common.geometry import to_geopandas, to_geopolars
from electoralyze.common.testing.region_fixture import (
    RegionMocked,
    create_fake_regions,
    read_true_geometry,
    read_true_metadata,
)
from electoralyze.region.redistribute import redistribute
from electoralyze.region.redistribute.mapping import (
    _create_intersection_area_mapping,
    _get_remaining_area,
    get_region_mapping_base,
)
from polars import testing

In [2]:
region.Federal2022

electoralyze.region.regions.federal_2022.Federal2022

In [3]:
region.SA1_2021

electoralyze.region.regions.SA1_2021.SA1_2021

In [4]:
region.Federal2022.geometry

federal_2022,geometry
str,binary
"""adelaide""","b""\x01\x03\x00\x00\xa0\xe6\x10\x00\x00\x01\x00\x00\x00W\x00\x00\x00u\x8e\x01\xd9kTa@\x82\xfd\xd7\xb9ioA\xc0\x00\x00\x00\x00\x00\x00\x00\x00\xd0\x98I\xd4\x0bTa@\x0a\xd68\x9b\x8epA\xc0\x00\x00\x00""…"
"""aston""","b""\x01\x03\x00\x00\xa0\xe6\x10\x00\x00\x01\x00\x00\x00\xd0\x02\x00\x00\xfc\x8e\xe1\xb1\x1f+b@|\xed\x99%\x01\xeeB\xc0\x00\x00\x00\x00\x00\x00\x00\x00;8\xd8\x9b\x18+b@-\xe8\xbd1\x04\xeeB\xc0\x00\x00\x00""…"
"""ballarat""","b""\x01\x03\x00\x00\xa0\xe6\x10\x00\x00\x01\x00\x00\x00D\x09\x00\x00J\xb2\x0eGW\xfaa@p\xb071$\x07C\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x05\xc0x\x06\x8d\xfaa@\xfa\xd1p\xca\xdc\x02C\xc0\x00\x00\x00""…"
"""banks""","b""\x01\x03\x00\x00\xa0\xe6\x10\x00\x00\x01\x00\x00\x00\x9b\x00\x00\x00\xab\x07\xccC&\xe4b@\x96\xaf\xcb\xf0\x9f\xfc@\xc0\x00\x00\x00\x00\x00\x00\x00\x006#\x83\xdc\xc5\xe3b@\xfc\xdf\x11\x15\xaa\xfd@\xc0\x00\x00\x00""…"
"""barker""","b""\x01\x06\x00\x00\xa0\xe6\x10\x00\x00\x1b\x00\x00\x00\x01\x03\x00\x00\x80\x01\x00\x00\x00M\x0d\x00\x00wj.7\x18\xa0a@\xa3\x05h[\xcd\x02A\xc0\x00\x00\x00\x00\x00\x00\x00\x00\xa0PO\x1f\x01\xa0a@r7""…"
…,…
"""werriwa""","b""\x01\x03\x00\x00\xa0\xe6\x10\x00\x00\x01\x00\x00\x00\x1d\x01\x00\x00*\x8d\x98\xd9g\xddb@\xf7\xe67L4\xf8@\xc0\x00\x00\x00\x00\x00\x00\x00\x00\x1a\x17\x0e\x84d\xddb@G\xc8@\x9e]\xf8@\xc0\x00\x00\x00""…"
"""whitlam""","b""\x01\x06\x00\x00\xa0\xe6\x10\x00\x00\x02\x00\x00\x00\x01\x03\x00\x00\x80\x01\x00\x00\x00\xd9\x04\x00\x00y\x95\xb5M\xf1\xdcb@\x17a\x8ariLA\xc0\x00\x00\x00\x00\x00\x00\x00\x00\xaf\xb1KT\xef\xdcb@u\x1e""…"
"""wide_bay""","b""\x01\x03\x00\x00\xa0\xe6\x10\x00\x00\x01\x00\x00\x00d\x0e\x00\x00\x8b\xe1\xea\x00\x88+c@\x01L\x198\xa0\x019\xc0\x00\x00\x00\x00\x00\x00\x00\x00\xf7\xb1\x82\xdf\x86+c@6\x93o\xb6\xb9\x019\xc0\x00\x00\x00""…"


In [5]:
region.SA1_2021.geometry.head(100)

SA1_2021,geometry
i64,binary
10102100701,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\xc9\x02\x00\x00]\x16V\xf6\x80\xbcb@`j""\xe5\x88\x8bA\xc0K\x07\x81\xbe\x89\xbcb@\x98\xe7[Du\x8bA\xc0\xdc\x8d\x1c;\x92\xbcb@\xa9X\x94""…"
10102100702,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x02\x00\x00\x00\x08\x01\x00\x00Y\xdc\xe9\xa6~\xb7b@\x9e\xfb\xe0\x81\x0c\xafA\xc0\xb5Q\xabKL\xb8b@y\x87\xa0!}\xafA\xc0\xdc\xeb1\x0bW\xb8b@A\xb6\x8d""…"
10102100703,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\x1a\x00\x00\x00\x0f\xf0N\x9b\xf5\xb8b@\xd60\xf1\x91g\xb8A\xc0u\xc3C\xa7\xf2\xb8b@G8\xcb\xb4+\xb8A\xc0B6\xc1\xac\xfb\xb8b@^p\xe5""…"
10102100704,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\x1e\x00\x00\x00\x95\x92\x10<}\xb9b@7J\xbb\x9f\x09\xb9A\xc0\xca1\xac\xee\x89\xb9b@j\x94\xfc\x020\xb8A\xc0RI\xbet\x8b\xb9b@\xc1\xe3\xb4""…"
10102100705,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00""\x00\x00\x00\xdc\xbc\x02\x88p\xb9b@\xfe\x05l#y\xb9A\xc0g\x10R\x1cr\xb9b@X.\x07\x8cy\xb9A\xc0\x93\xecfnt\xb9b@Pc\x1f""…"
…,…
10102101221,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\x10\x00\x00\x005[\xf6\xe7\x8d\xa6b@\xcb\x87\xf1u\x05\xaeA\xc0\xbbC\x9a\xa8\x94\xa6b@\xb2K\xa5Z\xfb\xadA\xc0F\xfd\x19\xd2\x99\xa6b@\x03\xaf\xc8""…"
10102101222,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\x0e\x00\x00\x00=\xdaAU\x91\xa6b@s_\xc0h)\xb1A\xc0\x0d`\xa0T\x94\xa6b@\xbe\xa4\xab\x17\xf1\xb0A\xc0\xc3\xae@\xbb\xbe\xa6b@@O\x8a""…"
10102101223,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\x13\x00\x00\x00\x91\x96\xa2KN\xa6b@y:\\xa0\x8c\xb1A\xc0\x9a\xba\xda\xe8V\xa6b@\x02\x17\xdd@\xe1\xb0A\xc0\x00&\x14\xe0d\xa6b@r\xdd\xf7""…"


In [6]:
from electoralyze.region.redistribute.mapping import _get_intersection_area

In [38]:
geometry_from = region.Federal2022.geometry
geometry_to = region.SA1_2021.geometry

# Using overlay

In [39]:
intersections = geometry_from.pipe(to_geopandas).overlay(geometry_to.pipe(to_geopandas), how="intersection")
intersection_area_new = (
    intersections.assign(intersection_area=lambda df: df["geometry"].area).drop("geometry", axis=1).pipe(pl.DataFrame)
)
intersection_area_new

/tmp/ipykernel_17058/1710801322.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersection_area_new = intersections.assign(intersection_area = lambda df: df['geometry'].area).drop("geometry", axis = 1).pipe(pl.DataFrame)


federal_2022,SA1_2021,intersection_area
str,i64,f64
"""adelaide""",40101100101,0.000014
"""adelaide""",40101100102,0.000007
"""adelaide""",40101100103,0.000009
"""adelaide""",40101100104,0.000008
"""adelaide""",40101100105,0.000009
…,…,…
"""wright""",31701145412,1.6867e-7
"""wright""",31701145415,1.9773e-7
"""wright""",31701145521,3.4680e-7


# other methods

In [40]:
geometry_combined: gpd.GeoDataFrame = (
    geometry_from.pipe(to_geopandas)
    .rename(columns={"geometry": "geometry_from"})
    .merge(geometry_to.pipe(to_geopandas).rename(columns={"geometry": "geometry_to"}), how="cross")
)
geometry_combined

intersection_area = (
    geometry_combined.assign(intersection_area=lambda df: df["geometry_from"].intersection(df["geometry_to"]).area)
    .drop(["geometry_from", "geometry_to"], axis=1)
    .loc[lambda df: df["intersection_area"] > 0]
    .pipe(pl.DataFrame)
)
intersection_area

/tmp/ipykernel_17058/2829525813.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometry_combined.assign(intersection_area=lambda df: df["geometry_from"].intersection(df["geometry_to"]).area)


federal_2022,SA1_2021,intersection_area
str,i64,f64
"""adelaide""",40101100101,0.000014
"""adelaide""",40101100102,0.000007
"""adelaide""",40101100103,0.000009
"""adelaide""",40101100104,0.000008
"""adelaide""",40101100105,0.000009
…,…,…
"""wright""",31701145412,1.6867e-7
"""wright""",31701145415,1.9773e-7
"""wright""",31701145521,3.4680e-7


In [41]:
pl.testing.assert_frame_equal(
    intersection_area_new,
    intersection_area,
)

### Multi threading

In [10]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial

import polars as pl


def calculate_intersection(row):
    """Calculate intersection area for a single row."""
    return row["geometry_from"].intersection(row["geometry_to"]).area


def parallel_intersection(df, max_workers=None):
    """
    Calculate intersection areas using parallel processing.

    Args:
        df: Input DataFrame with geometry_from and geometry_to columns
        max_workers: Number of threads to use. If None, uses default ThreadPoolExecutor value

    Returns
    -------
        Polars DataFrame with intersection areas
    """
    # Convert to pandas for easier row iteration if not already
    if isinstance(df, pl.DataFrame):
        df = df.to_pandas()

    # Create ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Calculate all intersection areas in parallel
        intersection_areas = list(executor.map(calculate_intersection, (row for _, row in df.iterrows())))

    # Create result DataFrame
    result_df = df.drop(["geometry_from", "geometry_to"], axis=1)
    result_df["intersection_area"] = intersection_areas

    # Convert to Polars DataFrame
    return pl.from_pandas(result_df)


# Usage example:
intersection_area = parallel_intersection(geometry_combined)

# If you want to specify the number of workers:
# intersection_area = parallel_intersection(geometry_combined, max_workers=4)